In [51]:
import pandas as pd

# Paso 1: Leer el CSV y establecer la primera columna como índice
df = pd.read_csv('data/similarity_matrix2.csv', index_col=0)

# Paso 2: Crear el diccionario de mapeo
labels = df.index.tolist()
mapping_dict = {label: idx for idx, label in enumerate(labels, start=1)}  # Empezamos en 1

# Paso 3: Reemplazar las etiquetas con números
df.rename(index=mapping_dict, inplace=True)
df.rename(columns=mapping_dict, inplace=True)

# Paso 4: Verificar el resultado
print(df.head())

        1         2         3         4         5         6         7    \
1  1.000000  0.497972  0.134061  0.065794  0.541140  0.183011  0.135995   
2  0.497972  1.000000 -0.057346  0.058898  0.259512  0.223909  0.197520   
3  0.134061 -0.057346  1.000000 -0.109140  0.348754 -0.053662  0.340687   
4  0.065794  0.058898 -0.109140  1.000000  0.106825  0.234263 -0.007710   
5  0.541140  0.259512  0.348754  0.106825  1.000000  0.129674  0.290657   

        8         9         10   ...       789       790       791       792  \
1  0.164164  0.107141 -0.079068  ...  0.115910  0.223875  0.287844  0.028263   
2  0.280954  0.205832  0.280593  ...  0.222299 -0.010720  0.192699  0.241937   
3 -0.248189  0.371167 -0.230586  ...  0.214892  0.569593  0.355426  0.004944   
4  0.372907  0.284071  0.253789  ...  0.344647  0.097781  0.125714  0.326580   
5 -0.009174  0.209874 -0.115949  ...  0.082730  0.371860  0.385915  0.108855   

        793       794       795       796       797       798  
1  0

In [53]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

# Descargar los 'stopwords' en inglés
nltk.download('stopwords')

# Leer el CSV
df_targets = pd.read_csv('data/targets_con_palabras_clave_agrupadas.csv')

# Obtener la lista de stopwords en inglés
stop_words = set(stopwords.words('english'))

def procesar_palabras_clave(palabras_clave):
    # Separar las frases por comas y eliminar espacios extra
    frases = [frase.strip() for frase in palabras_clave.split(',')]
    
    frases_procesadas = []
    for frase in frases:
        # Convertir a minúsculas
        frase = frase.lower()
        
        # Eliminar puntuación
        frase = frase.translate(str.maketrans('', '', string.punctuation))
        
        # Tokenizar la frase en palabras
        palabras = frase.split()
        
        # Eliminar stopwords
        palabras_filtradas = [palabra for palabra in palabras if palabra not in stop_words]
        
        # Unir las palabras filtradas de nuevo en una frase
        frase_filtrada = ' '.join(palabras_filtradas)
        
        # Agregar la frase procesada a la lista
        frases_procesadas.append(frase_filtrada)
    
    # Unir todas las frases procesadas en una cadena separada por comas
    return ', '.join(frases_procesadas)

# Aplicar la función a la columna y crear una nueva columna con los resultados
df_targets['palabras_clave_procesadas'] = df_targets['palabras_clave_concatenadas'].apply(procesar_palabras_clave)

# Mostrar las primeras filas del DataFrame
print(df_targets.head())

# Nota: No se guarda el DataFrame modificado en un nuevo CSV según lo solicitado.


   goal_id target_id                                             target  \
0        1       1.1  By 2030, eradicate extreme poverty for all peo...   
1        1       1.2  By 2030, reduce at least by half the proportio...   
2        1       1.3  Implement nationally appropriate social protec...   
3        1       1.4  By 2030, ensure that all men and women, in par...   
4        1       1.5  By 2030, build the resilience of the poor and ...   

                         palabras_clave_concatenadas  \
0                       poor and vulnerable, poverty   
1                                            poverty   
2                      vulnerable, social protection   
3  equal rights to economic resources, human righ...   
4         vulnerable, poor and vulnerable, resilient   

                           palabras_clave_procesadas  
0                           poor vulnerable, poverty  
1                                            poverty  
2                      vulnerable, social prote

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sgsr_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
import os
# Crear el directorio 'data' si no existe
output_dir = 'data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def mapear_frases_a_indices(palabras_clave, mapping_dict, missing_words_set):
    # Separar las frases por comas y eliminar espacios extra
    frases = [frase.strip() for frase in palabras_clave.split(',')]
    
    indices = []
    for frase in frases:
        if frase in mapping_dict:
            indices.append(str(mapping_dict[frase]))
        else:
            missing_words_set.add(frase)
    
    # Unir los índices en una cadena separada por comas
    return ', '.join(indices)

# Inicializar un conjunto para almacenar las palabras faltantes
missing_words = set()

# Aplicar la función a la columna y crear una nueva columna con los índices mapeados
df_targets['indices_mapeados'] = df_targets['palabras_clave_procesadas'].apply(
    lambda x: mapear_frases_a_indices(x, mapping_dict, missing_words)
)

# Definir la ruta para el archivo de palabras faltantes
missing_words_file = os.path.join(output_dir, 'missing_words.txt')

# Escribir las palabras faltantes en el archivo
with open(missing_words_file, 'w', encoding='utf-8') as file:
    for palabra in sorted(missing_words):
        file.write(f"{palabra}\n")

# Definir la ruta para el archivo CSV final
final_csv_path = os.path.join(output_dir, 'targets_con_palabras_clave_mapeadas.csv')

# Guardar el DataFrame en el archivo CSV
df_targets.to_csv(final_csv_path, index=False)

print(f"Archivo CSV guardado en: {final_csv_path}")
print(f"Palabras faltantes guardadas en: {missing_words_file}")

Archivo CSV guardado en: data\targets_con_palabras_clave_mapeadas.csv
Palabras faltantes guardadas en: data\missing_words.txt


In [63]:
df_targets

,goal_id,target_id,target,palabras_clave_concatenadas,palabras_clave_procesadas,indices_mapeados
0,1,1.1,"By 2030, eradicate extreme poverty for all peo...","poor and vulnerable, poverty","poor vulnerable, poverty","536, 275"
1,1,1.2,"By 2030, reduce at least by half the proportio...",poverty,poverty,275
2,1,1.3,Implement nationally appropriate social protec...,"vulnerable, social protection","vulnerable, social protection","194, 720"
3,1,1.4,"By 2030, ensure that all men and women, in par...","equal rights to economic resources, human righ...","equal rights economic resources, human rights,...","642, 583, 500, 45, 773"
4,1,1.5,"By 2030, build the resilience of the poor and ...","vulnerable, poor and vulnerable, resilient","vulnerable, poor vulnerable, resilient","194, 536, 368"
...,...,...,...,...,...,...
164,17,17.15,Respect each country’s policy space and leader...,"poverty eradication, development assistance, g...","poverty eradication, development assistance, g...","193, 667, 295, 293"
165,17,17.16,Enhance the global partnership for sustainable...,"publicprivate partnerships, global partnership...","publicprivate partnerships, global partnership...","295, 335, 293, 623"
166,17,17.17,"Encourage and promote effective public, public...",civil society partnerships,civil society partnerships,526
167,17,17.18,"By 2020, enhance capacity-building support to ...","least developed countries, small island develo...","least developed countries, small island develo...","16, 485, 293, 621, 740, 623"


In [82]:
import pandas as pd
import numpy as np
import os

# Asegúrate de haber leído previamente tus DataFrames:
# df_similarity: Matriz de similitud mapeada
# df_targets: DataFrame con 'target_id' y 'indices_mapeados'

# Supongamos que ya has leído y procesado los DataFrames como en los pasos anteriores
# df_similarity = pd.read_csv('similarity_matrix2.csv', index_col=0)
# df_targets = pd.read_csv('data/targets_con_palabras_clave_mapeadas.csv')

# Asegúrate de que los índices de df_similarity sean enteros
df.index = df.index.astype(int)
df.columns = df.columns.astype(int)

# Obtener la lista única de targets
target_list = list(set(df_targets['target_id'].tolist()))

# Crear un DataFrame vacío para almacenar los resultados
df_graph = pd.DataFrame(index=target_list, columns=target_list, dtype=float)

# Función para convertir la cadena de índices a una lista de enteros
def convertir_indices(ind_str):
    if pd.isna(ind_str):
        return []
    return [int(x.strip()) for x in ind_str.split(',')]

# Iterar sobre cada par de targets
for tg1 in target_list:
    # Obtener los índices para tg1
    ind1_str = df_targets[df_targets['target_id'] == tg1]['indices_mapeados'].values
    ind1 = convertir_indices(ind1_str[0]) if len(ind1_str) > 0 else []
    
    for tg2 in target_list:
        # Obtener los índices para tg2
        ind2_str = df_targets[df_targets['target_id'] == tg2]['indices_mapeados'].values
        ind2 = convertir_indices(ind2_str[0]) if len(ind2_str) > 0 else []
        
        # Inicializar suma y contador
        suma_similitudes = 0
        total_pares = 0
        
        # Calcular la suma de las similitudes para todos los pares de índices
        for x1 in ind1:
            for x2 in ind2:
                if x1 in df.index and x2 in df.columns:
                    suma_similitudes += df.loc[x1, x2]
                    total_pares += 1
                else:
                    print(f"Advertencia: Índices {x1} o {x2} no existen en la matriz de similitud.")
        
        # Calcular el promedio si hay al menos un par
        if total_pares > 0:
            promedio = suma_similitudes / total_pares
        else:
            promedio = np.nan  # O puedes asignar otro valor por defecto, como 0
        
        # Asignar el promedio al DataFrame
        df_graph.loc[tg1, tg2] = promedio

    # Opcional: Imprimir progreso
    print(f"Procesado target {tg1} ({target_list.index(tg1)+1}/{len(target_list)})")

# Mostrar el DataFrame resultante
print("\nDataFrame de Grafos de Similitud:")
print(df_graph)

# Guardar el DataFrame en un archivo CSV
output_dir = 'data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

final_csv_path_graph = os.path.join(output_dir, 'nuevo_dataframe.csv')
df_graph.to_csv(final_csv_path_graph)

print(f"\nNuevo DataFrame guardado en: {final_csv_path_graph}")


Procesado target 12.c (1/169)
Procesado target 11.3 (2/169)
Procesado target 5.2 (3/169)
Procesado target 3.c (4/169)
Procesado target 5.6 (5/169)
Procesado target 17.16 (6/169)
Procesado target 15.6 (7/169)
Procesado target 17.5 (8/169)
Procesado target 4.a (9/169)
Procesado target 15.c (10/169)
Procesado target 4.b (11/169)
Procesado target 16.9 (12/169)
Procesado target 1.a (13/169)
Procesado target 16.6 (14/169)
Procesado target 6.6 (15/169)
Procesado target 11.2 (16/169)
Procesado target 12.5 (17/169)
Procesado target 17.19 (18/169)
Procesado target 9.5 (19/169)
Procesado target 16.1 (20/169)
Procesado target 17.3 (21/169)
Procesado target 9.c (22/169)
Procesado target 10.6 (23/169)
Procesado target 14.4 (24/169)
Procesado target 10.a (25/169)
Procesado target 15.3 (26/169)
Procesado target 13.1 (27/169)
Procesado target 16.2 (28/169)
Procesado target 17.12 (29/169)
Procesado target 17.15 (30/169)
Procesado target 13.2 (31/169)
Procesado target 14.3 (32/169)
Procesado target 17.13